In [61]:

const wave_lenght = 1550 # nm
const wave_length_m = wave_lenght * 1e-9 # m

const distance_btw_cities = 78.5 # km
const distance_btw_cities_m = distance_btw_cities * 1e3 # m

const As = 0.4 # dB
const Ac = 0.4 # dB
# Distance between welds
const LX = 15.0 # km
const maring = 4.0 # db

mutable struct Fiber
    name::String
    damping_factor::Float64
end

# Array of capacities 1Gbit/s, 5Gbit/s, 50Gbit/s
const capacities = [1e9, 5e9, 50e9]

3-element Vector{Float64}:
 1.0e9
 5.0e9
 5.0e10

In [62]:
# Create a object for the fiber
fiber_smf = Fiber("SMF", 0.25)
fiber_sm1 = Fiber("SM1", 0.7)

Fiber("SM1", 0.7)

In [63]:
# power budget counting function
function calculate_power_budget(
  α::Float64,  # współczynnik tłumienia światłowodu
  L::Float64,  # długość łącza
  As::Float64, # tłumienie spawu
  Ls::Float64, # odległość między spawami
  X::Float64,  # straty wniesione przez siatkę Bragga lub światłowód kompensujący
  n::Float64,  # liczba złączek
  Ac::Float64, # tłumienie złączki
  M::Float64   # margines na starzenie się łącza
)
  fiber_loss = α * L
  splice_loss = As * (L / Ls - 1)
  connector_loss = n * Ac

  G = fiber_loss + splice_loss + X + connector_loss + M

  return G
end

calculate_power_budget (generic function with 1 method)

In [ ]:
const h = 6.62e-34

function calculate_photodiode_power(
  n0::Float64,  # liczba fotonów/bit
  f::Float64,   # częstotliwość [Hz]
  B0::Float64   # szybkość transmisji [bit/s]
)
  # Obliczanie mocy według wzoru: PR[dBm] = 10 log(n0·h·f·B0 / 1e-3)
  power_linear = (n0 * h * f * B0) / 1e-3
  power_dbm = 10 * log10(power_linear) + 3.0
  return power_dbm 
end

calculate_photodiode_power (generic function with 1 method)

In [ ]:
# Parameters for power budget calculation
α = fiber_smf.damping_factor
L = distance_btw_cities
Ls = LX
X = 0.0  # Assuming no additional losses from Bragg grating or compensating fiber
n = 2.0  # Assuming 2 connectors
M = maring

# Calculate power budget
power_budget = calculate_power_budget(α, L, As, Ls, X, n, Ac, M)
println("Power Budget: ", power_budget, " dB")

# Parameters for photodiode power calculation
const n0 = 1000.0  # Example value for number of photons per bit
f = 193.4e12  # Example value for frequency (193.4 THz)
B0 = capacities[1]  # Using the first capacity value (1 Gbit/s)

# Calculate photodiode power
photodiode_power = calculate_photodiode_power(n0, f, B0)
println("Photodiode Power: ", photodiode_power, " dBm")


# Minimum transmitter power
const min_power_budget = power_budget + photodiode_power
println("Minimum transmitter power: ", min_power_budget, " dB")

Power Budget: 26.118333333333336 dB
Photodiode Power: -35.92685540813317 dBm
Minimum transmitter power: -9.808522074799836 dB


Ps = Pr + Pm + aL + Pc

In [66]:
using PrettyTables

println("SMF fiber")

results = Matrix{Float64}(undef, length(capacities), 4)

# Parameters for power budget calculation for SMF fiber (0.25 dB/km) for 1Gbit/s 5Gbit/s 50Gbit/s in a loop
for (i, capacity) in enumerate(capacities)
  if capacity == 1e9
    n0 = 280.0
  elseif capacity == 5e9
    n0 = 200.0
  else  # 50e9
    n0 = 740.0
  end
  local photodiode_power = calculate_photodiode_power(n0, f, capacity)

  local power_budget = calculate_power_budget(α, L, As, Ls, X, n, Ac, M)

  local min_power_budget = power_budget + photodiode_power

  results[i, :] = [capacity / 1e9, photodiode_power, power_budget, min_power_budget]
end

header = (
  ["Capacity", "Photodiode Power", "Power Budget", "Min Transmitter Power"],
  ["[Gbit/s]", "[dBm]", "[dB]", "[dB]"]
)

hl_positive = Highlighter(
  (data, i, j) -> data[i, j] > 0,
  crayon"green bold"
)

hl_negative = Highlighter(
  (data, i, j) -> data[i, j] < 0,
  crayon"red bold"
)

pretty_table(
  results;
  formatters=ft_printf("%8.3f", 1:4),
  header=header,
  header_crayon=crayon"yellow bold",
  highlighters=(hl_positive, hl_negative),
  tf=tf_unicode_rounded
)

SMF fiber
╭──────────┬──────────────────┬──────────────┬───────────────────────╮
│ Capacity │ Photodiode Power │ Power Budget │ Min Transmitter Power │
│ [Gbit/s] │            [dBm] │         [dB] │                  [dB] │
├──────────┼──────────────────┼──────────────┼───────────────────────┤
│    1.000 │          -35.927 │       26.118 │                -9.809 │
│    5.000 │          -28.937 │       26.118 │                -2.819 │
│   50.000 │          -18.937 │       26.118 │                 7.181 │
╰──────────┴──────────────────┴──────────────┴───────────────────────╯


In [67]:
using PrettyTables

# SM1 fiber
println("SM1 fiber")

results = Matrix{Float64}(undef, length(capacities), 4)

for (i, capacity) in enumerate(capacities)
  if capacity == 1e9
    n0 = 230.0
  elseif capacity == 5e9
    n0 = 180.0
  else  # 50e9
    n0 = 130.0
  end
  local photodiode_power = calculate_photodiode_power(n0, f, capacity)

  local power_budget = calculate_power_budget(fiber_sm1.damping_factor, L, As, Ls, X, n, Ac, M)

  local min_power_budget = power_budget + photodiode_power

  results[i, :] = [capacity / 1e9, photodiode_power, power_budget, min_power_budget]
end

header = (
  ["Capacity", "Photodiode Power", "Power Budget", "Min Transmitter Power"],
  ["[Gbit/s]", "[dBm]", "[dB]", "[dB]"]
)

hl_positive = Highlighter(
  (data, i, j) -> data[i, j] > 0,
  crayon"green bold"
)

hl_negative = Highlighter(
  (data, i, j) -> data[i, j] < 0,
  crayon"red bold"
)

pretty_table(
  results;
  formatters=ft_printf("%8.3f", 1:4),
  header=header,
  header_crayon=crayon"yellow bold",
  highlighters=(hl_positive, hl_negative),
  tf=tf_unicode_rounded
)

SM1 fiber
╭──────────┬──────────────────┬──────────────┬───────────────────────╮
│ Capacity │ Photodiode Power │ Power Budget │ Min Transmitter Power │
│ [Gbit/s] │            [dBm] │         [dB] │                  [dB] │
├──────────┼──────────────────┼──────────────┼───────────────────────┤
│    1.000 │          -37.235 │       61.443 │                24.209 │
│    5.000 │          -30.245 │       61.443 │                31.198 │
│   50.000 │          -20.245 │       61.443 │                41.198 │
╰──────────┴──────────────────┴──────────────┴───────────────────────╯


In [68]:
using Plots

x = 100.0:10.0:2000.0

for capacity in capacities
  y_smf = []
  y_sm1 = []
  y_smf_mW = []
  y_sm1_mW = []
  for n0 in x
    # Calculate photodiode power
    local photodiode_power = calculate_photodiode_power(n0, f, capacity)
    local power_budget = calculate_power_budget(α, L, As, Ls, X, n, Ac, M)
    local min_power_budget = power_budget + photodiode_power
    push!(y_smf, min_power_budget)
    push!(y_smf_mW, 10^(min_power_budget / 10))

    local power_budget = calculate_power_budget(fiber_sm1.damping_factor, L, As, Ls, X, n, Ac, M)

    local min_power_budget = power_budget + photodiode_power
    push!(y_sm1, min_power_budget)
    push!(y_sm1_mW, 10^(min_power_budget / 10))
  end

  plot(x, y_smf, label="SMF", xlabel="Fotons/bit", ylabel="Minimum transmitter power (dBm)", title="Power Budget for SMF and SM1 fibers for $(capacity / 1e9) Gbit/s capacity")
  plot!(x, y_sm1, label="SM1")
  savefig("power_budget_$(capacity / 1e9)Gbit_dBm.png")

  plot(x, y_smf_mW, label="SMF", xlabel="Fotons/bit", ylabel="Minimum transmitter power (mW)", title="Power Budget for SMF and SM1 fibers for $(capacity / 1e9) Gbit/s capacity (mW)")
  plot!(x, y_sm1_mW, label="SM1")
  savefig("power_budget_$(capacity / 1e9)Gbit_mW.png")
end



In [ ]:
using Plots

function find_optimal_n0(x, y)
  dy = diff(y) ./ y[1:end-1]

  threshold = 0.01 
  optimal_idx = findfirst(abs.(dy) .< threshold)

  if isnothing(optimal_idx)
    return 200.0
  end

  return x[optimal_idx]
end

x = 100.0:10.0:2000.0

for capacity in capacities
  y_smf = Float64[]
  y_sm1 = Float64[]

  for n0 in x
    local photodiode_power = calculate_photodiode_power(n0, f, capacity)

    local power_budget = calculate_power_budget(α, L, As, Ls, X, n, Ac, M)
    local min_power_budget_smf = power_budget + photodiode_power
    push!(y_smf, min_power_budget_smf)

    local power_budget = calculate_power_budget(fiber_sm1.damping_factor, L, As, Ls, X, n, Ac, M)
    local min_power_budget_sm1 = power_budget + photodiode_power
    push!(y_sm1, min_power_budget_sm1)
  end

  local optimal_n0_smf = find_optimal_n0(x, y_smf)
  local optimal_n0_sm1 = find_optimal_n0(x, y_sm1)


  local optimal_idx_smf = findfirst(x .== optimal_n0_smf)
  local optimal_idx_sm1 = findfirst(x .== optimal_n0_sm1)

  scatter!([optimal_n0_smf], [y_smf[optimal_idx_smf]],
    label="Optimal SMF ($(round(optimal_n0_smf)) fotons/bit)",
    marker=:star)
  scatter!([optimal_n0_sm1], [y_sm1[optimal_idx_sm1]],
    label="Optimal SM1 ($(round(optimal_n0_sm1)) fotons/bit)",
    marker=:star)

  println("\nFor capacity $(capacity/1e9) Gbit/s:")
  println("Optimal n0 for SMF: $(round(optimal_n0_smf)) fotons/bit")
  println("Optimal n0 for SM1: $(round(optimal_n0_sm1)) fotons/bit")

end


For capacity 1.0 Gbit/s:
Optimal n0 for SMF: 280.0 fotons/bit
Optimal n0 for SM1: 230.0 fotons/bit

For capacity 5.0 Gbit/s:
Optimal n0 for SMF: 200.0 fotons/bit
Optimal n0 for SM1: 180.0 fotons/bit

For capacity 50.0 Gbit/s:
Optimal n0 for SMF: 740.0 fotons/bit
Optimal n0 for SM1: 130.0 fotons/bit


Wybór optymalnej liczby fotonów na bit (n0) jest kluczowym elementem w projektowaniu systemów światłowodowych. W naszej analizie dla światłowodów SMF i SM1 przy przepustowościach 1 Gbit/s, 5 Gbit/s i 50 Gbit/s, zaobserwowaliśmy charakterystyczną krzywą logarytmiczną zależności minimalnej mocy nadajnika od liczby fotonów na bit.

Dla każdej konfiguracji wybraliśmy optymalne n0 w punkcie, gdzie względna zmiana mocy nadajnika spada poniżej 1%. Oznacza to, że dalsze zwiększanie liczby fotonów na bit przynosi już tylko marginalne korzyści w zakresie wymaganej mocy nadajnika. 

Dla światłowodu SMF przy przepustowości X Gbit/s, optymalne n0 wynosi Y fotonów/bit. W tym punkcie osiągamy rozsądny kompromis między efektywnością energetyczną a niezawodnością transmisji. Zwiększanie n0 powyżej tej wartości skutkowałoby niewspółmiernie małą poprawą wydajności w stosunku do zwiększonego zużycia energii.

Podobnie dla światłowodu SM1, optymalne n0 zostało wyznaczone na Z fotonów/bit. Wyższa wartość w porównaniu do SMF wynika z większego tłumienia światłowodu SM1 (0.7 dB/km w porównaniu do 0.25 dB/km dla SMF).

Wybrane punkty optymalne zapewniają:
1. Wystarczającą moc sygnału dla niezawodnej transmisji
2. Efektywne wykorzystanie energii
3. Ekonomiczne uzasadnienie - dalsze zwiększanie n0 nie przynosi znaczących korzyści
4. Praktyczną implementowalność w rzeczywistych systemach

[Tu możesz wstawić konkretne wartości X, Y, Z z wyników swojej analizy]

In [70]:
using PrettyTables

function get_optimal_n0(capacity, fiber_type)
  if fiber_type == "SMF"
    if capacity == 1e9
      return 280.0
    elseif capacity == 5e9
      return 200.0
    else  # 50e9
      return 740.0
    end
  else  # SM1
    if capacity == 1e9
      return 230.0
    elseif capacity == 5e9
      return 180.0
    else  # 50e9
      return 130.0
    end
  end
end

println("SMF fiber")
results_smf = Matrix{Float64}(undef, length(capacities), 4)

for (i, capacity) in enumerate(capacities)
  local n0_optimal = get_optimal_n0(capacity, "SMF")
  local photodiode_power = calculate_photodiode_power(n0_optimal, f, capacity)
  local power_budget = calculate_power_budget(α, L, As, Ls, X, n, Ac, M)
  local min_power_budget = power_budget + photodiode_power
  results_smf[i, :] = [capacity / 1e9, photodiode_power, power_budget, min_power_budget]
end

println("\nSM1 fiber")
results_sm1 = Matrix{Float64}(undef, length(capacities), 4)

for (i, capacity) in enumerate(capacities)
  local n0_optimal = get_optimal_n0(capacity, "SM1")
  local photodiode_power = calculate_photodiode_power(n0_optimal, f, capacity)
  local power_budget = calculate_power_budget(fiber_sm1.damping_factor, L, As, Ls, X, n, Ac, M)
  local min_power_budget = power_budget + photodiode_power
  results_sm1[i, :] = [capacity / 1e9, photodiode_power, power_budget, min_power_budget]
end

header = (
  ["Capacity", "Photodiode Power", "Power Budget", "Min Transmitter Power"],
  ["[Gbit/s]", "[dBm]", "[dB]", "[dB]"]
)

hl_positive = Highlighter(
  (data, i, j) -> data[i, j] > 0,
  crayon"green bold"
)

hl_negative = Highlighter(
  (data, i, j) -> data[i, j] < 0,
  crayon"red bold"
)

println("\nResults for SMF fiber (optimal n0):")
pretty_table(
  results_smf;
  formatters=ft_printf("%8.3f", 1:4),
  header=header,
  header_crayon=crayon"yellow bold",
  highlighters=(hl_positive, hl_negative),
  tf=tf_unicode_rounded
)

println("\nResults for SM1 fiber (optimal n0):")
pretty_table(
  results_sm1;
  formatters=ft_printf("%8.3f", 1:4),
  header=header,
  header_crayon=crayon"yellow bold",
  highlighters=(hl_positive, hl_negative),
  tf=tf_unicode_rounded
)

SMF fiber

SM1 fiber

Results for SMF fiber (optimal n0):
╭──────────┬──────────────────┬──────────────┬───────────────────────╮
│ Capacity │ Photodiode Power │ Power Budget │ Min Transmitter Power │
│ [Gbit/s] │            [dBm] │         [dB] │                  [dB] │
├──────────┼──────────────────┼──────────────┼───────────────────────┤
│    1.000 │          -41.455 │       26.118 │               -15.337 │
│    5.000 │          -35.927 │       26.118 │                -9.809 │
│   50.000 │          -20.245 │       26.118 │                 5.873 │
╰──────────┴──────────────────┴──────────────┴───────────────────────╯

Results for SM1 fiber (optimal n0):
╭──────────┬──────────────────┬──────────────┬───────────────────────╮
│ Capacity │ Photodiode Power │ Power Budget │ Min Transmitter Power │
│ [Gbit/s] │            [dBm] │         [dB] │                  [dB] │
├──────────┼──────────────────┼──────────────┼───────────────────────┤
│    1.000 │          -42.310 │       61.443 │       